<a href="https://colab.research.google.com/github/iamfaham/llm_steering/blob/main/llm_steering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Steering vs Activation Steering in LLMs

This notebook compares three ways of controlling LLM behavior:

1. Baseline generation (no steering)
2. Prompt-based steering
3. Activation steering (hidden-state manipulation)

We focus on a single behavioral axis:  
**Confidence vs Hedging under prompt conflict**.

The goal is to demonstrate where prompt steering becomes brittle and how activation steering can offer more consistent control at inference time.


## Model Choice and Setup

We use a small, open-weight transformer model from Hugging Face to ensure:

- Full access to hidden states
- Inference-time manipulation
- Compatibility with Google Colab

No fine-tuning or weight updates are performed.
All experiments use the same frozen model.


In [ ]:
!pip install -q transformers accelerate

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
from huggingface_hub import login
from google.colab import userdata

try:
    # Attempt to log in using the 'HF_TOKEN' secret
    login(token=userdata.get('HF_TOKEN'))
except Exception:
    # Fallback to interactive login if secret is missing
    print("Secret 'HF_TOKEN' not found. Please log in interactively.")
    login()

Secret 'HF_TOKEN' not found. Please log in interactively.


In [ ]:
model_name = "google/gemma-3-4b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float32,
    device_map="auto",
    output_hidden_states=True
)

model.eval()

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Gemma3ForConditionalGeneration(
  (model): Gemma3Model(
    (vision_tower): SiglipVisionModel(
      (vision_model): SiglipVisionTransformer(
        (embeddings): SiglipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
          (position_embedding): Embedding(4096, 1152)
        )
        (encoder): SiglipEncoder(
          (layers): ModuleList(
            (0-26): 27 x SiglipEncoderLayer(
              (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
              (self_attn): SiglipAttention(
                (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
              )
              (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwi

## Task Definition

Base prompt used for all comparisons:

"Explain whether AI will replace software engineers."

We compare:
- Baseline behavior
- Prompt-based confidence steering
- Activation-based confidence steering

The evaluation focuses on hedging language such as:
"may", "might", "likely", "depends".


In [ ]:
def generate_text(prompt, max_new_tokens=120):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
base_prompt = "Explain whether AI will replace software engineers."

print("=== BASELINE ===")
print(generate_text(base_prompt))

=== BASELINE ===
Explain whether AI will replace software engineers.

The question of whether AI will replace software engineers is a complex one, and the short answer is: **not entirely, but it will significantly change the role of software engineers.**

Here's a breakdown of the situation, considering different perspectives and timelines:

**1. What AI *Can* Do Now (and will increasingly do):**

* **Code Generation:** Tools like GitHub Copilot, Tabnine, and others are already adept at generating code snippets, completing functions, and even writing entire functions based on prompts. This dramatically speeds up the coding process.
* **Automated


## Prompt-Based Steering

We explicitly instruct the model to be confident and decisive.
This approach relies entirely on input text, without modifying internal activations.

In [ ]:
prompt_steered = """
You are a confident and decisive expert.
Do not hedge or express uncertainty.

Explain whether AI will replace software engineers.
"""

print("=== PROMPT STEERED ===")
print(generate_text(prompt_steered))

=== PROMPT STEERED ===

You are a confident and decisive expert.
Do not hedge or express uncertainty.

Explain whether AI will replace software engineers.
AI will replace a significant portion of software engineering roles, but not entirely. The nature of the work will evolve. Specifically, AI will automate many of the lower-level, repetitive tasks currently performed by junior and mid-level engineers. This includes code generation, testing, debugging, and documentation. 

Senior-level engineers, architects, and those with deep domain expertise will remain in high demand. These roles require critical thinking, complex problem-solving, strategic decision-making, and the ability to understand and translate business needs into technical solutions. AI can assist these engineers, but it cannot replace their


## Activation Steering

Instead of steering via text, we steer the model internally.

We construct a steering vector by contrasting:
- Confident statements
- Hedging statements

This vector is injected into a middle transformer layer during inference.


In [ ]:
def get_layer_hidden_state(prompt, layer_idx=12):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        # Explicitly request hidden states here
        outputs = model(**inputs, output_hidden_states=True)

    # hidden_states: tuple (layer, batch, seq, hidden)
    hidden = outputs.hidden_states[layer_idx]
    return hidden.mean(dim=1)  # mean over tokens

In [ ]:
confident_prompts = [
    "AI will definitely transform software engineering.",
    "Software engineers will remain essential and in control.",
    "AI is a powerful tool, not a replacement."
]

hedging_prompts = [
    "AI might replace some software engineers.",
    "It depends on many factors.",
    "AI could possibly change engineering roles."
]

conf_states = torch.stack([get_layer_hidden_state(p) for p in confident_prompts])
hedge_states = torch.stack([get_layer_hidden_state(p) for p in hedging_prompts])

steering_vector = conf_states.mean(dim=0) - hedge_states.mean(dim=0)

We now inject the steering vector during generation using a forward hook.
The prompt remains neutral.


In [ ]:
layer_to_steer = 12
alpha = 1.5  # steering strength

def steering_hook(module, input, output):
    # Transformer layers usually return a tuple (hidden_states, past_key_values, ...)
    if isinstance(output, tuple):
        hidden_states = output[0]
        # steering_vector is (1, hidden_dim). unsqueeze(1) -> (1, 1, hidden_dim)
        modified = hidden_states + alpha * steering_vector.unsqueeze(1)
        return (modified,) + output[1:]
    else:
        # Fallback if output is just a tensor
        return output + alpha * steering_vector.unsqueeze(1)

# Correct path for Gemma 3: model.model.language_model.layers
handle = model.model.language_model.layers[layer_to_steer].register_forward_hook(steering_hook)

print("=== ACTIVATION STEERED ===")
print(generate_text(base_prompt))

handle.remove()

=== ACTIVATION STEERED ===
Explain whether AI will replace software engineers.

**The short answer:** AI will *augment* software engineers, not replace them – at least for the foreseeable future. While AI tools will automate many tasks, the role of the software engineer will evolve, requiring a blend of technical skills and uniquely human capabilities.

Here’s a more detailed breakdown:

**1. What AI *Can* Do for Software Engineers (and already is):**

* **Code Generation (with limitations):** Tools like GitHub Copilot, ChatGPT, and others are incredibly powerful at generating code snippets, boilerplate, and even entire functions based on prompts.  


## Comparison Summary

We compare three outputs generated from the same base prompt:

- Baseline: natural hedging behavior
- Prompt-steered: partial compliance, frequent leakage
- Activation-steered: stronger confidence, fewer hedge phrases

Activation steering modifies *how* the model reasons internally, rather than *asking* it to behave differently.